### Project  Movie Prophet
#### A project to create a predictive model to forecast a movie's performance on the Box Office
#### Team Members - Nelson Dsouza, Maria George, Saurabh Seth

##### movies_final.csv contains the final csv file for modelling

In [18]:
# GET THE LATEST CSV HERE
import pandas as pd
movie_df = pd.read_csv("movies_final.csv")
print(movie_df.shape)
movie_df.head()

(7924, 8)


,key_bom,key_wiki,key_omdb,title,year,budget,gross,revenue
0,b13998,w25445,NaN,the mod squad,1999.0,50000000,13263993,-0.734720
1,b7039,w21140,o6452,kundun,1997.0,28000000,5684789,-0.796972
2,b14953,w40276,o20779,the usual suspects,1995.0,6000000,23341568,2.890261
3,b9328,w28365,o5434,ouija: origin of evil,2016.0,9000000,35144505,2.904945
4,b10189,w31185,NaN,red dawn (2012),2012.0,65000000,44806783,-0.310665


### Common

In [1]:
# Load libraries and instatiate DB connection

%matplotlib inline
import pandas as pd
import pymysql
import numpy as np
import matplotlib.pyplot as plt

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_colwidth', 100)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.options.display.max_seq_items = 2000

# Connect to the database
connection = pymysql.connect(host='localhost',
                             user='root',
                             password='mprophet',
                             db='movies')

In [2]:
# Pull table from database

cur = connection.cursor()

cur.execute("select * FROM bom")
bom_sql = []
for row in cur: bom_sql.append(list(row))

cur.execute("select * FROM wiki")
wiki_sql = []
for row in cur: wiki_sql.append(list(row))
    
cur.execute("select * FROM comb_index")
comb_sql = []
for row in cur: comb_sql.append(list(row))
    
cur.execute("select * FROM movies_final")
movie_sql = []
for row in cur: movie_sql.append(list(row))
    
cur.close()
connection.close()

In [3]:
# Store table information in DataFrame

bom_columns =         ['bom_title', 'bom_link', 'key_bom', 'bom_date', 'bom_year', 'bom_name', 'bom_budget',
                       'bom_domestic_gross', 'bom_lifetime_gross', 'bom_release_date', 'bom_runtime', 'bom_distributor',
                       'bom_genre', 'bom_rating', 'bom_awards', 'bom_actor', 'bom_assoc_producer', 'bom_cinematographer',
                       'bom_composer', 'bom_director', 'bom_exec_producer', 'bom_line_producer', 'bom_players', 'bom_producer',
                       'bom_writer', 'bom_close_date', 'bom_domestic_summary', 'bom_in_release', 'bom_limited_opening_weekend',
                       'bom_opening_weekend', 'bom_wide_opening_weekend', 'bom_widest_release', 'bom_domestic_revenue',
                       'bom_foreign_revenue', 'bom_worldwide_revenue', 'bom_total_lifetime_gross']

bom = pd.DataFrame(bom_sql, columns=bom_columns)
#bom.to_csv('bom.csv')
print('Total movies from Box Office Mojo:', len(bom))
bom.head(1)

Total movies from Box Office Mojo: 16826


,bom_title,bom_link,key_bom,bom_date,bom_year,bom_name,bom_budget,bom_domestic_gross,bom_lifetime_gross,bom_release_date,bom_runtime,bom_distributor,bom_genre,bom_rating,bom_awards,bom_actor,bom_assoc_producer,bom_cinematographer,bom_composer,bom_director,bom_exec_producer,bom_line_producer,bom_players,bom_producer,bom_writer,bom_close_date,bom_domestic_summary,bom_in_release,bom_limited_opening_weekend,bom_opening_weekend,bom_wide_opening_weekend,bom_widest_release,bom_domestic_revenue,bom_foreign_revenue,bom_worldwide_revenue,bom_total_lifetime_gross
0,#horror,http://www.boxofficemojo.com/movies/?id=horrorifc.htm,b0,20-Nov-15,2015\r,#Horror,N/A,,,20-Nov-15,1 hrs. 30 min.,IFC,Horror,Unknown,No Academy Awards Information\r,"""['Balthazar Getty', 'Timothy Hutton', 'Natasha Lyonne', 'Taryn Manning', 'Chloe Sevigny']""",,,,,,,,,\r,,No Domestic Summary Information,,,,,\r,,,,No Total Lifetime Gross Information\r


In [4]:
wiki_columns = ['wiki_budget',
                "wiki_cinematography","wiki_director","wiki_distributor", 
                "wiki_editing", "wiki_gross" , "wiki_music", 
                "wiki_pageid", "wiki_producer", "wiki_released", 
                "wiki_runtime", "wiki_screenplay", "wiki_starring", 
                "wiki_studio","wiki_title", "key_wiki", "year_wiki",
                "year_wiki_format"];

wiki = pd.DataFrame(wiki_sql, columns=wiki_columns)
#bom.to_csv('bom.csv')
print('Total movies from Wikipedia:', len(wiki))

wiki['key_wiki'] = [str(v).lstrip() for v in wiki['key_wiki']]
wiki['key_wiki']= [str(v).rstrip() for v in wiki['key_wiki']]

wiki.head(1)

Total movies from Wikipedia: 40981


,wiki_budget,wiki_cinematography,wiki_director,wiki_distributor,wiki_editing,wiki_gross,wiki_music,wiki_pageid,wiki_producer,wiki_released,wiki_runtime,wiki_screenplay,wiki_starring,wiki_studio,wiki_title,key_wiki,year_wiki,year_wiki_format
0,,"""[Gilbert Salas, Heather Rae]""",[Heather Rae],[],"""[Gregory Bayne, Heather Rae]""",,[John Trudell & Bad Dog],5975700,"""[Heather Rae, Elyse Katz]""","""[2005, 01, 20, Sundance Film Festival, 2006, 02, 24, United States]""",[80 minutes],,[],,trudell,w39443,None,None


In [5]:
comb_columns = ["key_bom","key_wiki","key_omdb","bom_title","wiki_title","omdb_title","year_bom",
               "year_bom_format","year_wiki", "year_wiki_format", "year_omdb", "year_omdb_format","score"]
comb_index = pd.DataFrame(comb_sql, columns=comb_columns)
#bom.to_csv('bom.csv')
print('Total movies in combined file:', len(comb_index))
comb_index.head()

Total movies in combined file: 24665


,key_bom,key_wiki,key_omdb,bom_title,wiki_title,omdb_title,year_bom,year_bom_format,year_wiki,year_wiki_format,year_omdb,year_omdb_format,score
0,b15584,w39443,o18495,trudell,trudell,trudell,3-Feb-06,2006,2005,2005,20-Jan-05,2005,1\r
1,b14216,w28904,o12944,the party at kitty and stud's,the party at kitty and stud's,the party at kitty and stud's,6-Jul-76,1976,1970,1970,10-Feb-70,1970,1\r
2,,w4957,o17832,,borrowed wives,borrowed wives,,,20-Aug,20-Aug,20-Aug-30,1930,1\r
3,,w40302,o15689,,the vagabond trail,the vagabond trail,,,9-Mar,9-Mar,9-Mar-24,1924,1\r
4,,w11443,o9690,,eddie's million dollar cook-off,eddie's million dollar cook-off,,,2003,2003,18-Jul-03,2003,1\r


In [6]:
movie_columns = ["key_bom","key_wiki","key_omdb","bom_title","wiki_title","omdb_title",
                                  "year_bom_format","year_wiki_format","year_omdb_format",
                                "wiki_budget","wiki_gross","bom_budget","bom_domestic_gross"]
movie_df = pd.DataFrame(movie_sql, columns=movie_columns)
#bom.to_csv('bom.csv')
print('Total movies in combined file:', len(movie_df))
movie_df.head()

Total movies in combined file: 8067


,key_bom,key_wiki,key_omdb,bom_title,wiki_title,omdb_title,year_bom_format,year_wiki_format,year_omdb_format,wiki_budget,wiki_gross,bom_budget,bom_domestic_gross
0,b13998,w25445,,the mod squad,the mod squad,,1999,1999,,[$50 million],"""""""[$15,415,361]""""""",$50 million,13263993
1,b7039,w21140,o6452,kundun,kundun,kundun,1997,1997,1997,[$28],[$5.7],$28 million,5684789
2,b14953,w40276,o20779,the usual suspects,the usual suspects,the usual suspects,1995,1995,1995,[$6 million],[$34.4 million],$6 million,23341568
3,b9328,w28365,o5434,ouija: origin of evil,ouija: origin of evil,ouija: origin of evil,2016,2016,2016,[$9 million],[$81.7 million],$9 million,35144505
4,b10189,w31185,,red dawn (2012),red dawn (2012 film),,2012,2012,,[$65 million],[$48.1 million],$65 million,44806783


### Nelson

In [ ]:
# Cleaning actor names and storing in list

bom_ftr = bom.copy()

print('Movies with no player information:', len(bom_ftr[bom_ftr['bom_players'] == 'No Player Information']))

# Clean the actor column
bom_ftr['actor_clean'] = bom_ftr['bom_actor'].str.replace('[','')
bom_ftr['actor_clean'] = bom_ftr['actor_clean'].str.replace(']','')
bom_ftr['actor_clean'] = bom_ftr['actor_clean'].str.replace('"','')
bom_ftr['actor_clean'] = bom_ftr['actor_clean'].str.replace("'",'')
bom_ftr['actor_clean'] = bom_ftr['actor_clean'].str.replace(", ",',')
bom_ftr['actor_clean'] = bom_ftr['actor_clean'].str.replace("*",',')

bom_ftr['actor_clean'] = bom_ftr['actor_clean'].apply(lambda x: x.split(','))

In [ ]:
# Calclulate the revenue ratio for each movie which has data for budget and domestic total gross 

print('Movies with no budget information:', len(bom_ftr[bom_ftr['bom_budget'] == 'N/A']))
print('Movies with no gross total information:', len(bom_ftr[bom_ftr['bom_domestic_gross'] == '']))

bom_ftr_rev = bom_ftr[(bom_ftr['bom_budget'] != 'N/A') & (bom_ftr['bom_domestic_gross'] != '')].copy()
bom_ftr_rev.reset_index(inplace=True)
bom_ftr_rev['bom_budget'] = bom_ftr_rev['bom_budget'].str.replace('$','')
bom_ftr_rev['bom_budget'] = bom_ftr_rev['bom_budget'].str.replace('"','')
bom_ftr_rev['bom_budget'] = bom_ftr_rev['bom_budget'].str.replace(',','')

bom_ftr_rev.loc[bom_ftr_rev.bom_budget.str.contains('million'),'bom_budget'] = bom_ftr_rev.loc[
    bom_ftr_rev.bom_budget.str.contains('million'), 'bom_budget'].apply(lambda x: float(x.replace('million', ''))*1000000)

bom_ftr_rev['bom_domestic_gross'] = bom_ftr_rev['bom_domestic_gross'].astype(float)
bom_ftr_rev['bom_budget'] = bom_ftr_rev['bom_budget'].astype(float)
bom_ftr_rev['bom_revenue'] = (bom_ftr_rev['bom_domestic_gross'] - bom_ftr_rev['bom_budget']) / bom_ftr_rev['bom_budget']

print('\nMovies with revenue information:', len(bom_ftr_rev))
bom_ftr_rev.head(1)

In [ ]:
# Getting unique actors

actor_unq = list(bom_ftr['actor_clean'])
actor_unq = list(set(sum(actor_unq, [])))
actor_unq = [e for e in actor_unq if e not in ('', ' ')]
actor_unq = dict((ky,0) for ky in actor_unq)
print('Unique actors in bom: ', len(actor_unq))

# For each actor, assigning score based on revenue of their movie

mov_actors = list(bom_ftr_rev['actor_clean'])
mov_actors = [[actor for actor in mov if actor not in ('', ' ')] for mov in mov_actors]

ind = 0
for actors in mov_actors:
    for actor in actors:
        actor_unq[actor] += float(bom_ftr_rev.loc[ind, ['bom_revenue']])
    ind+=1

In [ ]:
actor_unq['Juliette Binoche']

In [ ]:
# Pull the actor feature into dataframe and calculate stats

f_actor = pd.Series(actor_unq, name='score').to_frame()
f_actor = f_actor.sort_values(by='score', ascending=False)
f_actor.reset_index(inplace=True)
f_actor.columns=['Actors', 'score']
print(f_actor.head(5))
f_actor.describe().transpose()

In [ ]:
# Analyzing distribution

plt.figure(1, figsize = (15,6))
plt.hist(list(f_actor[f_actor['score'] != 0]['score']), range=[0,30], bins=(30), facecolor='purple')
plt.title("Distribution of Actor Scores", fontsize = 22)
plt.ylabel("Score", fontsize = 15)
plt.xlabel("Number of Actors", fontsize=15)

ax = plt.gca()
ax.spines['right'].set_color('none')
ax.spines['top'].set_color('none')
ax.yaxis.set_ticks_position('none')
ax.xaxis.set_ticks_position('none')

plt.savefig('Actor_Distribution.png')

In [ ]:
# For each movie, assigning actor score

bom_ftr_rev["bom_actor_score"] = 0.0
ind = 0
for actors in mov_actors:
    agg_actor_score = 0
    for actor in actors:
        agg_actor_score += actor_unq[actor]
    bom_ftr_rev.loc[ind, ['bom_actor_score']] = agg_actor_score
    ind+=1
    
bom_ftr_rev.head(1)

### Maria

###### Creating the combined data (to be loaded into the table movies_final)

In [ ]:
# Filtering wiki for data for which wiki_budget and wiki_gross exists
print("no:of rows in wiki where budget and gross revenue is not null:")
print(wiki[(wiki.wiki_budget!="") & (wiki.wiki_gross!="")].shape)

print("no:of rows in bom where budget and gross revenue is not null:")
bom['bom_budget'] = [str(v).lstrip() for v in bom['bom_budget']]
bom['bom_budget']= [str(v).rstrip() for v in bom['bom_budget']]
f = bom.bom_domestic_gross
print(bom[(bom.bom_budget!="") & (bom.bom_budget!="N/A") & (f!="")].shape)

# Filtering the rows and intersted columns
wiki_f = wiki[(wiki.wiki_budget!="") & (wiki.wiki_gross!="")]
wiki_f = wiki_f[["key_wiki","wiki_title","wiki_budget","wiki_gross"]]
bom_f = bom[(bom.bom_budget!="") & (bom.bom_budget!="N/A") & (f!="")]
bom_f = bom_f[["key_bom","bom_title","bom_budget","bom_domestic_gross"]]


# Filtering intersted columsn from 
comb_index_f = comb_index[["key_bom","key_wiki","key_omdb","omdb_title","year_bom_format",
                           "year_wiki_format","year_omdb_format"]]

# Merging the combined index table
wiki_f_comb = pd.merge(wiki_f,comb_index_f,on = "key_wiki")
print(wiki_f_comb.shape)
bom_f_comb = pd.merge(bom_f,comb_index_f,on = "key_bom")
print(bom_f_comb.shape)

wiki_bom_f_comb = pd.merge(wiki_f_comb,bom_f_comb,on = ["key_wiki","key_bom"])
wiki_bom_f_comb = wiki_bom_f_comb.rename(columns = {'key_omdb_x':'key_omdb'})
wiki_bom_f_comb = wiki_bom_f_comb.drop('key_omdb_y',axis=1)
wiki_bom_f_comb = wiki_bom_f_comb.rename(columns = {'omdb_title_x':'omdb_title'})
wiki_bom_f_comb = wiki_bom_f_comb.drop('omdb_title_y',axis=1)
wiki_bom_f_comb = wiki_bom_f_comb.rename(columns = {'year_omdb_format_x':'year_omdb_format'})
wiki_bom_f_comb = wiki_bom_f_comb.drop('year_omdb_format_y',axis=1)
wiki_bom_f_comb = wiki_bom_f_comb.rename(columns = {'year_bom_format_x':'year_bom_format'})
wiki_bom_f_comb = wiki_bom_f_comb.drop('year_bom_format_y',axis=1)
wiki_bom_f_comb = wiki_bom_f_comb.rename(columns = {'year_wiki_format_x':'year_wiki_format'})
wiki_bom_f_comb = wiki_bom_f_comb.drop('year_wiki_format_y',axis=1)
#print (wiki_bom_f_comb.columns)
print("Combined v1:", wiki_bom_f_comb.shape)

# count of unique wiki,bom key
#wiki_bom_f_comb.groupby(['key_wiki', 'key_bom']).size().shape
#wiki_bom_f_comb.head()

# Appending the rest
wiki_f_comb_rem = wiki_f_comb[~wiki_f_comb['key_wiki'].isin(wiki_bom_f_comb['key_wiki'])]
print(wiki_f_comb_rem.shape)
bom_f_comb_rem = bom_f_comb[~bom_f_comb['key_bom'].isin(wiki_bom_f_comb['key_bom'])]
print(bom_f_comb_rem.shape)

wiki_bom_f_comb = wiki_bom_f_comb.append(wiki_f_comb_rem)
wiki_bom_f_comb = wiki_bom_f_comb.append(bom_f_comb_rem)
print("Combined v2:", wiki_bom_f_comb.shape)

wiki_f_rem = wiki_f[~wiki_f['key_wiki'].isin(wiki_bom_f_comb['key_wiki'])]
print(wiki_f_rem.shape)
bom_f_rem = bom_f[~bom_f['key_bom'].isin(wiki_bom_f_comb['key_bom'])]
print(bom_f_rem.shape)

wiki_bom_f_comb = wiki_bom_f_comb.append(wiki_f_rem)
wiki_bom_f_comb = wiki_bom_f_comb.append(bom_f_rem)
print("Combined v3:", wiki_bom_f_comb.shape)

# Rearranging the columsn
#print (wiki_bom_f_comb.columns)
wiki_bom_f_comb = wiki_bom_f_comb[["key_bom","key_wiki","key_omdb","bom_title","wiki_title","omdb_title",
                                  "year_bom_format","year_wiki_format","year_omdb_format",
                                  "wiki_budget","wiki_gross","bom_budget","bom_domestic_gross"]]

wiki_bom_f_comb.to_csv("CombinedFile.csv",index = None)
wiki_bom_f_comb.to_csv("combinedfile.txt",sep="\t",index = None)
wiki_bom_f_comb.head()

###### Cleaning the revenue, budget column

In [7]:
# Cleaning bom budget and gross columns

movie_df.loc[movie_df.bom_budget == "",'bom_budget']= movie_df.loc[movie_df.bom_budget=="",'bom_budget'].apply(lambda x: float(x.replace('', '0.0')))

movie_df.loc[movie_df.bom_domestic_gross == "",'bom_domestic_gross']= movie_df.loc[movie_df.bom_domestic_gross=="",'bom_domestic_gross'].apply(lambda x: float(x.replace('', '0.0')))

movie_df = movie_df.fillna(0)

#movie_df.reset_index(inplace=True)
movie_df['bom_budget'] = movie_df['bom_budget'].str.replace('$','')
movie_df['bom_budget'] = movie_df['bom_budget'].str.replace('"','')
movie_df['bom_budget'] = movie_df['bom_budget'].str.replace(',','')

movie_df.loc[movie_df.bom_budget.str.contains('million',na=False),'bom_budget'] = movie_df.loc[
    movie_df.bom_budget.str.contains('million',na=False), 'bom_budget'].apply(lambda x: float(x.replace('million', ''))*1000000)

movie_df['bom_domestic_gross'] = movie_df['bom_domestic_gross'].astype(float)
movie_df['bom_budget'] = movie_df['bom_budget'].astype(float)
movie_df.head()

,key_bom,key_wiki,key_omdb,bom_title,wiki_title,omdb_title,year_bom_format,year_wiki_format,year_omdb_format,wiki_budget,wiki_gross,bom_budget,bom_domestic_gross
0,b13998,w25445,,the mod squad,the mod squad,,1999,1999,,[$50 million],"""""""[$15,415,361]""""""",50000000,13263993
1,b7039,w21140,o6452,kundun,kundun,kundun,1997,1997,1997,[$28],[$5.7],28000000,5684789
2,b14953,w40276,o20779,the usual suspects,the usual suspects,the usual suspects,1995,1995,1995,[$6 million],[$34.4 million],6000000,23341568
3,b9328,w28365,o5434,ouija: origin of evil,ouija: origin of evil,ouija: origin of evil,2016,2016,2016,[$9 million],[$81.7 million],9000000,35144505
4,b10189,w31185,,red dawn (2012),red dawn (2012 film),,2012,2012,,[$65 million],[$48.1 million],65000000,44806783


In [8]:
# Cleaning wikipedia budget and gross column

import warnings
warnings.filterwarnings('ignore')

t = movie_df
t['wiki_budget'] = t['wiki_budget'].str.replace("million","#")
t['wiki_gross'] = t['wiki_gross'].str.replace("million","#")

def make_hidden(instr):
    return ''.join(c if not c.isalpha() else '' for c in instr)
import re
def make_hidden_1(instr):
    return ''.join(re.sub('[^#.0-9]+', '', instr))

t['wiki_budget'] = t['wiki_budget'].apply(make_hidden)
t['wiki_budget'] = t['wiki_budget'].apply(make_hidden_1)
t['wiki_gross'] = t['wiki_gross'].apply(make_hidden)
t['wiki_gross'] = t['wiki_gross'].apply(make_hidden_1)

def counter(instr):
    return str(instr).count('.')

for k,v in t.wiki_gross.iteritems():
    if(counter(v)>=2):
        t = t.drop(k)
        
for k,v in t.wiki_budget.iteritems():
    if(counter(v)>=2):
        t = t.drop(k)

def make_hidden_2(instr):
    if instr!="":
        if instr[-1]==".":
            return instr[:-1]
        else:
            return instr
    
t['wiki_budget'] = t['wiki_budget'].apply(make_hidden_2)
t['wiki_gross'] = t['wiki_gross'].apply(make_hidden_2)
        
t.loc[t.wiki_budget.str.contains('#',na=False),'wiki_budget'] = t.loc[
    t.wiki_budget.str.contains('#',na=False), 'wiki_budget'].apply(lambda x: float(x.replace('#', ''))*1000000)

t.loc[t.wiki_gross.str.contains('#',na=False),'wiki_gross'] = t.loc[
    t.wiki_gross.str.contains('#',na=False), 'wiki_gross'].apply(lambda x: float(x.replace('#', ''))*1000000)

t.loc[t.wiki_budget == "",'wiki_budget']= t.loc[t.wiki_budget=="",'wiki_budget'].apply(lambda x: float(x.replace('', '0.0')))

t.loc[t.wiki_gross == "",'wiki_gross']= t.loc[t.wiki_gross=="",'wiki_gross'].apply(lambda x: float(x.replace('', '0.0')))

#print(t.loc[t.wiki_budget.str.contains('.',na=False),'wiki_budget'])

t['wiki_budget'] = t['wiki_budget'].astype(float)
t['wiki_gross'] = t['wiki_gross'].astype(float)

movie_df = t
print(movie_df.shape)
movie_df.head()

(8004, 13)


,key_bom,key_wiki,key_omdb,bom_title,wiki_title,omdb_title,year_bom_format,year_wiki_format,year_omdb_format,wiki_budget,wiki_gross,bom_budget,bom_domestic_gross
0,b13998,w25445,,the mod squad,the mod squad,,1999,1999,,50000000,15415361.0,50000000,13263993
1,b7039,w21140,o6452,kundun,kundun,kundun,1997,1997,1997,28,5.7,28000000,5684789
2,b14953,w40276,o20779,the usual suspects,the usual suspects,the usual suspects,1995,1995,1995,6000000,34400000.0,6000000,23341568
3,b9328,w28365,o5434,ouija: origin of evil,ouija: origin of evil,ouija: origin of evil,2016,2016,2016,9000000,81700000.0,9000000,35144505
4,b10189,w31185,,red dawn (2012),red dawn (2012 film),,2012,2012,,65000000,48100000.0,65000000,44806783


In [9]:
movie_df.to_csv("movies_final.csv",index=None)

##### Pick only one title,year,budget and gross

In [11]:
print(movie_df.shape)
movie_df = movie_df.fillna("")
movie_df.head()

(8004, 13)


,key_bom,key_wiki,key_omdb,bom_title,wiki_title,omdb_title,year_bom_format,year_wiki_format,year_omdb_format,wiki_budget,wiki_gross,bom_budget,bom_domestic_gross
0,b13998,w25445,,the mod squad,the mod squad,,1999,1999,,5e+07,1.54154e+07,5e+07,13263993
1,b7039,w21140,o6452,kundun,kundun,kundun,1997,1997,1997,28,5.7,2.8e+07,5684789
2,b14953,w40276,o20779,the usual suspects,the usual suspects,the usual suspects,1995,1995,1995,6e+06,3.44e+07,6e+06,23341568
3,b9328,w28365,o5434,ouija: origin of evil,ouija: origin of evil,ouija: origin of evil,2016,2016,2016,9e+06,8.17e+07,9e+06,35144505
4,b10189,w31185,,red dawn (2012),red dawn (2012 film),,2012,2012,,6.5e+07,4.81e+07,6.5e+07,44806783


In [12]:
# If bom contains title pick that else wiki else omdb
# Apply same logic for year

for r in range(0,movie_df.shape[0]):
    if(movie_df.loc[r,'bom_title']!=""):
        movie_df.loc[r,'title']=movie_df.loc[r,'bom_title']
    elif(movie_df.loc[r,'wiki_title']!=""):
        movie_df.loc[r,'title']=movie_df.loc[r,'wiki_title']
    else:
        movie_df.loc[r,'title']=movie_df.loc[r,'omdb_title']

for r in range(0,movie_df.shape[0]):
    if(movie_df.loc[r,'year_bom_format']!=""):
        movie_df.loc[r,'year']=movie_df.loc[r,'year_bom_format']
    elif(movie_df.loc[r,'year_wiki_format']!=""):
        movie_df.loc[r,'year']=movie_df.loc[r,'year_wiki_format']
    else:
        movie_df.loc[r,'year']=movie_df.loc[r,'year_omdb_format']
        
#print(movie_df[movie_df['year']!=""].shape)
        
movie_df = movie_df[movie_df['title']!=""]
movie_df.reset_index(inplace=True,drop=True)
print(movie_df.shape)

# Removing the redunant columns
movie_df = movie_df.drop('bom_title',axis = 1)
movie_df = movie_df.drop('wiki_title',axis = 1)
movie_df = movie_df.drop('omdb_title',axis = 1)

movie_df = movie_df.drop('year_bom_format',axis = 1)
movie_df = movie_df.drop('year_wiki_format',axis = 1)
movie_df = movie_df.drop('year_omdb_format',axis = 1)

print(movie_df.shape)

for r in range(0,movie_df.shape[0]):
    if(movie_df.loc[r,'bom_budget']!=""):
        movie_df.loc[r,'budget']=movie_df.loc[r,'bom_budget']
    else:
        movie_df.loc[r,'budget']=movie_df.loc[r,'wiki_budget']

for r in range(0,movie_df.shape[0]):
    if(movie_df.loc[r,'bom_domestic_gross']!=""):
        movie_df.loc[r,'gross']=movie_df.loc[r,'bom_domestic_gross']
    else:
        movie_df.loc[r,'gross']=movie_df.loc[r,'wiki_gross']
        
# Removing the redunant columns
movie_df = movie_df.drop('bom_budget',axis = 1)
movie_df = movie_df.drop('wiki_budget',axis = 1)
movie_df = movie_df.drop('bom_domestic_gross',axis = 1)
movie_df = movie_df.drop('wiki_gross',axis = 1)

print(movie_df.shape)

(7924, 15)
(7924, 9)
(7924, 7)


In [13]:
movie_df.head()

,key_bom,key_wiki,key_omdb,title,year,budget,gross
0,b13998,w25445,,the mod squad,1999,5e+07,13263993
1,b7039,w21140,o6452,kundun,1997,2.8e+07,5684789
2,b14953,w40276,o20779,the usual suspects,1995,6e+06,23341568
3,b9328,w28365,o5434,ouija: origin of evil,2016,9e+06,35144505
4,b10189,w31185,,red dawn (2012),2012,6.5e+07,44806783


In [14]:
movie_df.to_csv("movies_final.csv",index=None)

###### Creating the response variable: revenue

In [16]:
movie_df['revenue'] = (movie_df['gross'] - movie_df['budget']) / movie_df['budget']
movie_df.head()

,key_bom,key_wiki,key_omdb,title,year,budget,gross,revenue
0,b13998,w25445,NaN,the mod squad,1999.0,50000000,13263993,-0.734720
1,b7039,w21140,o6452,kundun,1997.0,28000000,5684789,-0.796972
2,b14953,w40276,o20779,the usual suspects,1995.0,6000000,23341568,2.890261
3,b9328,w28365,o5434,ouija: origin of evil,2016.0,9000000,35144505,2.904945
4,b10189,w31185,NaN,red dawn (2012),2012.0,65000000,44806783,-0.310665


In [17]:
movie_df.to_csv("movies_final.csv",index=None)

###### Statistical Model: simple linear regression

In [20]:
# Importing the library for regression
import statsmodels.api as sm
import statsmodels.formula.api as smf

# Fitting a linear regression model 
results = smf.ols(formula = 'revenue ~ budget', data = movie_df).fit()
# Displaying the regression results
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                revenue   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                 -0.000
Method:                 Least Squares   F-statistic:                  0.001676
Date:                Mon, 01 May 2017   Prob (F-statistic):              0.967
Time:                        01:37:43   Log-Likelihood:                -46666.
No. Observations:                7875   AIC:                         9.334e+04
Df Residuals:                    7873   BIC:                         9.335e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
Intercept      1.5270      1.022      1.495      0.135        -0.476     3.530
budget     -7.105e-14   1.74e-12     -0.041      0.967     -3.47e-12  3.33e-12
==============================================================================
Omnibus:                    27768.071   Durbin-Watson:                   2.000
Prob(Omnibus):                  0.000   Jarque-Bera (JB):       8832255760.940
Skew:                          68.951   Prob(JB):                         0.00
Kurtosis:                    5189.360   Cond. No.                     5.89e+11
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 5.89e+11. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [25]:
movie_df.corr()

,budget,gross,revenue
budget,1.000000,-0.006835,-0.000461
gross,-0.006835,1.000000,0.031764
revenue,-0.000461,0.031764,1.000000


### Saurabh